In [1]:
import pandas as pd
from transformers import AutoTokenizer
import tqdm
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
# Assurez-vous que votre colonne cible ('type') est codée en entiers
from sklearn.preprocessing import LabelEncoder
import torch
from pytorch_lightning import LightningModule, Trainer
from datasets import Dataset
from datasets import DatasetDict



c:\Users\arthu\miniconda3\envs\mbti\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Charger les données depuis un fichier Parquet
data = pd.read_parquet('data/curated/curated_data.parquet')

# Afficher les premières lignes pour vérifier
print(data.head())

   type                                              posts
0  INFJ  ' enfp and intj moments sportscenter not top t...
1  ENTP  'i'm finding the lack of me in these posts ver...
2  INTP  'good one _____ of course, to which i say i kn...
3  INTJ  'dear intp, i enjoyed our conversation the oth...
4  ENTJ  'you're fired. that's another silly misconcept...


In [3]:
# Tokeniser les textes


In [4]:


from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

data['input_ids'] = data['posts'].apply(lambda x: tokenizer.encode(x, truncation=True, padding='max_length', max_length=512))

# Créer des attention masks (utile si vous utilisez des séquences de longueur variable)
data['attention_mask'] = data['input_ids'].apply(lambda x: [int(token_id > 0) for token_id in x])


encoder = LabelEncoder()
data['labels'] = encoder.fit_transform(data['type'])


In [5]:


# Convertir le DataFrame pandas en Dataset Hugging Face
dataset = Dataset.from_pandas(data[['input_ids', 'attention_mask', 'labels']])
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


In [6]:

# Diviser le dataset en ensembles d'entraînement et de test
train_test_split = dataset.train_test_split(test_size=0.2)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})


In [7]:
from transformers import AutoModelForSequenceClassification

num_labels = len(set(data['labels']))  # Le nombre de classes distinctes
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"]
)


In [9]:
trainer.train()


  0%|          | 3/1302 [03:49<27:54:59, 77.37s/it]